In [1]:
import fridge
import fridge.utilities.h5Interface as h5i
import fridge.utilities.reactorInterface as RI
import os

from scipy.interpolate import interp1d as interp
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import osbrain
from osbrain import run_nameserver
from osbrain import run_agent
from osbrain import logging
import numpy as np
import blackboard
import ka
import ka_rp as karp
import ka_br as kabr
import bb_sfr_opt
import time

# SFR Optimization

This example seeks to optimize a SFR design utilizing the blackboard framework.
We start first by taking a look at the SFR design data using the `h5` add `reactor` interfaces built into `FRDIGe` [cite].
For this we'll examine the objectives that will drive the optimization process: cycle length, plutonium mass, reactivity swing, and core average burnup.
Upon examination, we note that we have removed a significant portion of the design space by ensuring that the reactor design has a cycle length greater than 0 days.

In [3]:
h5_interface = h5i.h5Interface()
h5_interface.read_h5('/Users/ryanstewart/projects/sfr_database/SFR_DB.h5')

In [8]:
cycle_length_list = []
rx_swing_list = []
height = []
smear = []
pu_content = []
bu = []
pu_mass = []
for rx, core in h5_interface.h5file.items():
    try:
        rx_ = RI.reactorInterface(core)
        cycle_length = rx_.extrapolate_value('keff', 'time', 1.0)
        rx_swing = rx_.get_reactivity_swing(1.0, cycle_length)
        rx_swing_per_day = rx_swing/cycle_length * 30
        avg_bu = rx_.get_assembly_avg(cycle_length, 'burnup')
        avg_bu_per_day = avg_bu/cycle_length * 30
        pu = rx_.get_assembly_pu_mass() * 78 / 4
        if cycle_length > 180:
            height.append(rx_.rx['independent variables']['height'][0])
            smear.append(rx_.rx['independent variables']['smear'][0])
            pu_content.append(rx_.rx['independent variables']['pu_content'][0])
            cycle_length_list.append(cycle_length)
            rx_swing_list.append(rx_swing_per_day)
            bu.append(avg_bu_per_day)
            pu_mass.append(pu)
    except (KeyError, TypeError):
        print('didnt get {}'.format(core))

In [9]:
fig = px.scatter_3d(x=height, y=smear, z=cycle_length_list, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Cycle Length (days)','color':'Pu Fraction'})
fig.show()
fig2 = px.scatter_3d(x=height, y=smear, z=rx_swing_list, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Rx Swing (pcm)','color':'Pu Fraction'})
fig2.show()
fig3 = px.scatter_3d(x=height, y=smear, z=bu, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Burnup (GWd/MTU)','color':'Pu Fraction'})
fig3.show()
fig4 = px.scatter_3d(x=height, y=smear, z=pu_mass, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Pu Mass per Cycle (kg)','color':'Pu Fraction'})
fig4.show()

# Starting the Blackboard System

In [2]:
ns = run_nameserver()
bb = run_agent(name='bb', base=bb_sfr_opt.BbSfrOpt)
bb.set_attr(desired_results={'cycle length': (180, 720), 
                             'reactivity swing': (540, 950), 
                             'burnup': (3.0, 5.5), 
                             'pu mass': (0, 1891)})
bb.generate_sm()
bb.set_attr(required_agents=[karp.KaRpExplore,karp.KaRpExploit,kabr.KaBr_lvl2,kabr.KaBr_lvl3])
bb.connect_agent(karp.KaRpExplore, 'ka_rp_explore_1')
bb.connect_agent(karp.KaRpExplore, 'ka_rp_explore_2')
bb.connect_agent(karp.KaRpExploit, 'ka_rp_exploit')
bb.connect_agent(kabr.KaBr_lvl2, 'ka_br_lvl2')
bb.connect_agent(kabr.KaBr_lvl3, 'ka_br_lvl3')

Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:16592 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:16592
INFO [2020-05-08 18:22:03.308036] (bb): Connected agent ka_rp_explore_1 of agent type <class 'ka_rp.KaRpExplore'>
INFO [2020-05-08 18:22:03.359624] (bb): Connected agent ka_rp_explore_2 of agent type <class 'ka_rp.KaRpExplore'>
INFO [2020-05-08 18:22:03.413606] (bb): Connected agent ka_rp_exploit of agent type <class 'ka_rp.KaRpExploit'>
INFO [2020-05-08 18:22:03.461560] (bb): Connected agent ka_br_lvl2 of agent type <class 'ka_br.KaBr_lvl2'>
INFO [2020-05-08 18:22:03.511099] (bb): Connected agent ka_br_lvl3 of agent type <class 'ka_br.KaBr_lvl3'>
INFO [2020-05-08 18:22:04.527422] (bb): Selecting agent ka_rp_explore_1 (TV: 1) to execute (TE: 1)
INFO [2020-05-08 18:22:05.542632] (bb): Selecting agent ka_rp_explore_2 (TV: 2) to execute (TE: 2)
INFO [2020-05-08 18:22:06.557153] (bb): Selecting agent ka_rp_explore_1 (TV: 2) to execute (TE: 3)
INFO [2020-05-08 18:22:

ERROR [2020-05-08 18:23:15.704284] (ka_br_lvl2): An exception occurred while running! ('core_[57.85, 53.45, 0.92]')

|>>>>>>>>Traceback (most recent call last):
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1680, in run
|     self._loop()
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1208, in _loop
|     if self._iterate():
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1245, in _iterate
|     self._process_events(events)
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1282, in _process_events
|     self._process_single_event(socket)
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1296, in _process_single_event
|     self._process_sub_event(socket, address, data)
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbra

INFO [2020-05-08 18:23:16.686583] (bb): Selecting agent ka_rp_explore_2 (TV: 2) to execute (TE: 48)
INFO [2020-05-08 18:23:17.704298] (bb): Selecting agent ka_rp_explore_1 (TV: 2) to execute (TE: 49)
INFO [2020-05-08 18:23:18.722422] (bb): Selecting agent ka_br_lvl3 (TV: 3) to execute (TE: 50)
INFO [2020-05-08 18:23:19.734843] (bb): Selecting agent ka_rp_explore_2 (TV: 3) to execute (TE: 51)
INFO [2020-05-08 18:23:20.748993] (bb): Selecting agent ka_rp_explore_1 (TV: 3) to execute (TE: 52)
INFO [2020-05-08 18:23:21.757444] (bb): Selecting agent ka_br_lvl3 (TV: 3) to execute (TE: 53)
INFO [2020-05-08 18:23:22.769973] (bb): Selecting agent ka_rp_explore_2 (TV: 3) to execute (TE: 54)
INFO [2020-05-08 18:23:23.785371] (bb): Selecting agent ka_rp_explore_1 (TV: 3) to execute (TE: 55)
INFO [2020-05-08 18:23:24.799560] (bb): Selecting agent ka_rp_explore_2 (TV: 2) to execute (TE: 56)
INFO [2020-05-08 18:23:25.816596] (bb): Selecting agent ka_rp_explore_1 (TV: 2) to execute (TE: 57)
INFO [2020

INFO [2020-05-08 18:26:19.818916] (bb): Selecting agent ka_br_lvl2 (TV: 4) to execute (TE: 127)
INFO [2020-05-08 18:26:20.832636] (bb): Selecting agent ka_br_lvl3 (TV: 3) to execute (TE: 128)
INFO [2020-05-08 18:26:21.844704] (bb): Selecting agent ka_br_lvl2 (TV: 4) to execute (TE: 129)
INFO [2020-05-08 18:26:22.854245] (bb): No KA to execute, waiting to sends trigger again.
INFO [2020-05-08 18:26:22.856969] (bb): Writing blackboard to archive
INFO [2020-05-08 18:26:28.867129] (bb): Finished writing to archive
INFO [2020-05-08 18:26:29.897984] (bb): Selecting agent ka_rp_explore_1 (TV: 9) to execute (TE: 131)
INFO [2020-05-08 18:26:30.907152] (bb): Selecting agent ka_rp_explore_2 (TV: 11) to execute (TE: 132)
INFO [2020-05-08 18:26:31.919117] (bb): Selecting agent ka_rp_explore_1 (TV: 2) to execute (TE: 133)
INFO [2020-05-08 18:26:32.927938] (bb): No KA to execute, waiting to sends trigger again.
INFO [2020-05-08 18:26:33.940172] (bb): Selecting agent ka_br_lvl2 (TV: 4) to execute (TE:

ERROR [2020-05-08 18:26:39.037099] (ka_br_lvl2): An exception occurred while running! ('core_[63.68, 53.02, 0.73]')

|>>>>>>>>Traceback (most recent call last):
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1680, in run
|     self._loop()
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1208, in _loop
|     if self._iterate():
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1245, in _iterate
|     self._process_events(events)
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1282, in _process_events
|     self._process_single_event(socket)
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1296, in _process_single_event
|     self._process_sub_event(socket, address, data)
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbra

...

INFO [2020-05-08 18:26:46.232222] (bb): Selecting agent ka_rp_explore_1 (TV: 1) to execute (TE: 141)
INFO [2020-05-08 18:26:47.242071] (bb): Selecting agent ka_rp_explore_2 (TV: 3) to execute (TE: 142)
INFO [2020-05-08 18:26:48.256627] (bb): Selecting agent ka_br_lvl3 (TV: 3) to execute (TE: 143)
INFO [2020-05-08 18:26:49.271728] (bb): Selecting agent ka_rp_explore_1 (TV: 3) to execute (TE: 144)
INFO [2020-05-08 18:26:50.284858] (bb): No KA to execute, waiting to sends trigger again.
INFO [2020-05-08 18:26:51.308399] (bb): Selecting agent ka_rp_exploit (TV: 5) to execute (TE: 146)
INFO [2020-05-08 18:26:52.355592] (bb): Selecting agent ka_rp_explore_2 (TV: 5) to execute (TE: 147)
INFO [2020-05-08 18:26:53.370042] (bb): Selecting agent ka_rp_explore_1 (TV: 4) to execute (TE: 148)
INFO [2020-05-08 18:26:54.384050] (bb): Selecting agent ka_rp_explore_2 (TV: 2) to execute (TE: 149)
INFO [2020-05-08 18:26:55.393501] (bb): Selecting agent ka_rp_explore_1 (TV: 2) to execute (TE: 150)
INFO [20

ERROR [2020-05-08 18:27:11.248889] (ka_br_lvl2): An exception occurred while running! ('core_[70.64, 55.64, 0.35]')

|>>>>>>>>Traceback (most recent call last):
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1680, in run
|     self._loop()
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1208, in _loop
|     if self._iterate():
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1245, in _iterate
|     self._process_events(events)
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1282, in _process_events
|     self._process_single_event(socket)
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1296, in _process_single_event
|     self._process_sub_event(socket, address, data)
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbra

INFO [2020-05-08 18:27:11.732701] (bb): No KA to execute, waiting to sends trigger again.
INFO [2020-05-08 18:27:11.735010] (bb): Writing blackboard to archive
INFO [2020-05-08 18:27:17.752314] (bb): Finished writing to archive
INFO [2020-05-08 18:27:17.870442] (bb): Found agent type {} not connect. 
 Reconnecting agent.
INFO [2020-05-08 18:27:17.925944] (bb): Connected agent ka_br_lvl2 of agent type <class 'ka_br.KaBr_lvl2'>
INFO [2020-05-08 18:28:11.510674] (bb): Selecting agent ka_rp_explore_1 (TV: 5) to execute (TE: 161)
INFO [2020-05-08 18:28:12.525835] (bb): Selecting agent ka_rp_explore_2 (TV: 5) to execute (TE: 162)
INFO [2020-05-08 18:28:13.540839] (bb): Selecting agent ka_rp_exploit (TV: 5) to execute (TE: 163)
INFO [2020-05-08 18:28:14.572273] (bb): Selecting agent ka_rp_explore_1 (TV: 3) to execute (TE: 164)
INFO [2020-05-08 18:28:15.587039] (bb): Selecting agent ka_rp_explore_2 (TV: 3) to execute (TE: 165)
INFO [2020-05-08 18:28:16.600641] (bb): Selecting agent ka_br_lvl3 

ERROR [2020-05-08 18:28:18.681625] (ka_br_lvl2): An exception occurred while running! ('core_[68.14, 62.96, 0.14]')

|>>>>>>>>Traceback (most recent call last):
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1680, in run
|     self._loop()
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1208, in _loop
|     if self._iterate():
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1245, in _iterate
|     self._process_events(events)
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1282, in _process_events
|     self._process_single_event(socket)
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1296, in _process_single_event
|     self._process_sub_event(socket, address, data)
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbra

INFO [2020-05-08 18:28:19.642777] (bb): Selecting agent ka_rp_exploit (TV: 5) to execute (TE: 169)
INFO [2020-05-08 18:28:20.675066] (bb): Selecting agent ka_rp_explore_1 (TV: 3) to execute (TE: 170)
INFO [2020-05-08 18:28:20.678890] (bb): Writing blackboard to archive
INFO [2020-05-08 18:28:26.691033] (bb): Finished writing to archive
INFO [2020-05-08 18:28:26.806847] (bb): Found agent type {} not connect. 
 Reconnecting agent.
INFO [2020-05-08 18:28:26.860239] (bb): Connected agent ka_br_lvl2 of agent type <class 'ka_br.KaBr_lvl2'>
NS shut down.


In [11]:
#while not bb.get_attr('_complete'):
#    i=0
for i in range(10):
    bb.publish_trigger()
    time.sleep(1)
    bb.controller()
    bb.send_executor()
    bb.wait_for_ka()
    bb.determine_complete()
    print('Finished trigger event: {}'.format(bb.get_attr('_trigger_event')))


Finished trigger event: 161
Finished trigger event: 162
Finished trigger event: 163
Finished trigger event: 164
Finished trigger event: 165
Finished trigger event: 166
Finished trigger event: 167
Finished trigger event: 168
Finished trigger event: 169
Finished trigger event: 170


In [6]:
print(ns.agents())
#bblvl2 = ns.proxy('ka_br_lvl2')

['bb', 'ka_rp_explore_1', 'ka_rp_explore_2', 'ka_rp_exploit', 'ka_br_lvl2', 'ka_br_lvl3']


In [12]:
print(bblvl2.get_attr('_running'))

False
